# Introduction

This document serves to provide an orientation to differnet OpenSSL commands.

## Computing Environment

Some basic information about the environment in which the Nobeook is executing.

In [3]:
# What environment am I running in?
!echo "Kernel:" && uname -a && echo ""
!echo "MacOS Version:" && sw_vers && echo ""
!echo "OpenSSL Version:" && openssl version && echo ""
!echo "Python Version:" && python --version && echo ""



Kernel:
Darwin USWSA220038.local 23.6.0 Darwin Kernel Version 23.6.0: Wed Jul 31 20:49:39 PDT 2024; root:xnu-10063.141.1.700.5~1/RELEASE_ARM64_T6000 arm64

MacOS Version:
ProductName:		macOS
ProductVersion:		14.7
BuildVersion:		23H124

OpenSSL Version:
OpenSSL 3.3.1 4 Jun 2024 (Library: OpenSSL 3.3.1 4 Jun 2024)

Python Version:
Python 3.11.6



## Available RSA Key Sizes

Below are the exponents associated with given RSA key lengths. 

In [4]:
# Some possible RSA key lenths.
for exponent in range(7,16):
    print("2^{0:3d} = {1:5d} bits".format(exponent, 2**exponent))

2^  7 =   128 bits
2^  8 =   256 bits
2^  9 =   512 bits
2^ 10 =  1024 bits
2^ 11 =  2048 bits
2^ 12 =  4096 bits
2^ 13 =  8192 bits
2^ 14 = 16384 bits
2^ 15 = 32768 bits


Note, that 3072-bit (2048+1024) keys are not referenced in this list.  It is likely that 3072-bit is trying to strike a cost/performance balance between weak 2048-bit and high computation cost of 4096-bit RSA keys.  So where to 3072-bit RSA keys come from?

In [5]:
# RSA key sizes are often represented as exponents to 2.
print(f"2048-bit RSA keys are derived from 2^11 = {2**11}.")
print(f"Where does 3072-bit RSA keys come from?")
print(f"- 3072-bit RSA keys might be 2^11.584962501 = {2**11.584962501}.")
print(f"- Or simply 2048*1.5 = {2048*1.5}.")

2048-bit RSA keys are derived from 2^11 = 2048.
Where does 3072-bit RSA keys come from?
- 3072-bit RSA keys might be 2^11.584962501 = 3072.000000593755.
- Or simply 2048*1.5 = 3072.0.


## Available Elliptic Curves

Some resources on about elliptic curves applied to TLS protocol.   
* [RFC 8422, Section 5.1.1 Supported Elliptic Curves Extensions](https://www.rfc-editor.org/rfc/rfc8422.html#section-5.1.1) namely `secp256r1`, `secp384r1`, `secp521r1` and a few others.
* [AWS Security Blog](https://aws.amazon.com/blogs/security/) discusses [How to evaluate and use ECDSA certificates in AWS Certificate Manager](https://aws.amazon.com/blogs/security/how-to-evaluate-and-use-ecdsa-certificates-in-aws-certificate-manager/).  It is acknowledged this article discusses ECDSA and not EC. It was just an simple read on EC curve options.

For the sake of simplicity and scoping, will focus on `prime256v1` and `secp384r1` curves only.  The tool `openssl` offers support for several different elliptic curves as noted below.

In [6]:
# What are the EC Curves supported by OpenSSL?
!openssl ecparam -list_curves

  secp112r1 : SECG/WTLS curve over a 112 bit prime field
  secp112r2 : SECG curve over a 112 bit prime field
  secp128r1 : SECG curve over a 128 bit prime field
  secp128r2 : SECG curve over a 128 bit prime field
  secp160k1 : SECG curve over a 160 bit prime field
  secp160r1 : SECG curve over a 160 bit prime field
  secp160r2 : SECG/WTLS curve over a 160 bit prime field
  secp192k1 : SECG curve over a 192 bit prime field
  secp224k1 : SECG curve over a 224 bit prime field
  secp224r1 : NIST/SECG curve over a 224 bit prime field
  secp256k1 : SECG curve over a 256 bit prime field
  secp384r1 : NIST/SECG curve over a 384 bit prime field
  secp521r1 : NIST/SECG curve over a 521 bit prime field
  prime192v1: NIST/X9.62/SECG curve over a 192 bit prime field
  prime192v2: X9.62 curve over a 192 bit prime field
  prime192v3: X9.62 curve over a 192 bit prime field
  prime239v1: X9.62 curve over a 239 bit prime field
  prime239v2: X9.62 curve over a 239 bit prime field
  prime239v3: X9.62 curv

# X.509 Key Types

The goal of this section is to demonstrate the differences between message digests (or thumbprints) between two different (but related) data structures with X.509 certificates.  
* Thumbprint of the signed certificate.
* Thumbprint of the cryptographic public key.

In the following sections we use RSA and ECC X.509 key types to perform the following tasks.
1. Create an single private key.
2. Derive from this private key the corresponding private key.
3. Generate multiple certificate signing requesest based on the single private key.
4. Self-sign each signing request with the private key.
5. We then extract the SubjectPublicKeyInfo from each signing request and self-signed certificate.
6. Finally, we compare the SubjectPublicKeyeInfo across all the various exports.

File names follow a mixed combination of hyphens (`-`) to separate words within a given field (e.g. `private-key`) and underscores (`_`) to separate fields (e.g. `rsa_private-key`).  


## RSA

Resources
1. [openssl-asn1parse](https://docs.openssl.org/master/man1/openssl-asn1parse/) processing ASN.1 files
2. [openssl-pkey](https://docs.openssl.org/master/man1/openssl-pkey/) interacting with private keys
3. [openssl-req](https://docs.openssl.org/master/man1/openssl-req/) processing signing-requests
4. [openssl-rsa](https://docs.openssl.org/master/man1/openssl-rsa/) generating RSA private keys
5. [openssl-x509](https://docs.openssl.org/master/man1/openssl-x509/) processing X.509 certificates

### Generate RSA Private Key

For illustration purposes, let us create a single RSA private key. 

> Warning: Please do not use these keys in any production or live environment as they are intended for demonstration purposes only.  As a _demo_, you have no control over who may have viewed or obtain a copy of this key material making these private keys potentially _public_.

In [7]:
%%bash
# preamble
FOLDER="data/poc"
mkdir -p "${FOLDER}"
RSA_PRIVATE_KEY_PEM="${FOLDER}/rsa_private-key.pem"
RSA_PRIVATE_KEY_DER="${FOLDER}/rsa_private-key.der"
KEYLENGTH="2048" # Typical sizes: 2048, 3072, 4096`

# validate environment
echo "Environment:"
echo "  Working Folder: '${FOLDER}'"
echo "  Private Key PEM: '${RSA_PRIVATE_KEY_PEM}'"
echo "  Private Key DER: '${RSA_PRIVATE_KEY_DER}'"
echo "  Key Length: '${KEYLENGTH}'"
echo ""

# execute
echo "Generating RSA Private Key:"

# create the private key in PEM form.
openssl genrsa -out "${RSA_PRIVATE_KEY_PEM}" "${KEYLENGTH}"

# convert private key from PEM (ASCII) to DER (binary)
openssl rsa -inform PEM -in "${RSA_PRIVATE_KEY_PEM}" -outform DER -out "${RSA_PRIVATE_KEY_DER}"
echo ""

# valdiate result
echo "Files:"
ls -l -go "${FOLDER}"/rsa_private-key.*
echo ""

echo "File Types:"
file "${RSA_PRIVATE_KEY_PEM}" "${RSA_PRIVATE_KEY_DER}"
echo ""

echo "File Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_private-key.*

Environment:
  Working Folder: 'data/poc'
  Private Key PEM: 'data/poc/rsa_private-key.pem'
  Private Key DER: 'data/poc/rsa_private-key.der'
  Key Length: '2048'

Generating RSA Private Key:


writing RSA key



Files:
-rw-------@ 1   1218 Sep 20 12:39 data/poc/rsa_private-key.der
-rw-------@ 1   1704 Sep 20 12:39 data/poc/rsa_private-key.pem

File Types:
data/poc/rsa_private-key.pem: ASCII text
data/poc/rsa_private-key.der: data

File Digests (SHA256):
6da9ba4a1d09a6000faefbb83887e6f39c68655fc62bd7d5463858ba235bb364  data/poc/rsa_private-key.der
f406150d7421fb04827c8037c8101b41dca0e1699cdf8257001bad7f008a80b1  data/poc/rsa_private-key.pem


### Derive the RSA Public Key

The raw private key material itself does not contain the public key.  However we can derive the cryptographic public key material from the private key using one of the `openssl` command options.  
1. `openssl rsa -pubout ...`
2. `openssl pkey -pubout ...`

In [8]:
%%bash
# preamble
FOLDER="data/poc"
mkdir -p "${FOLDER}"
RSA_PRIVATE_KEY_PEM="${FOLDER}/rsa_private-key.pem"
RSA_PRIVATE_KEY_DER="${FOLDER}/rsa_private-key.der"


# validate environment
echo "Environment:"
echo "  RSA Private Key PEM: '${RSA_PRIVATE_KEY_PEM}'"
echo "  RSA Private Key DER: '${RSA_PRIVATE_KEY_DER}'"
echo ""

# execute
echo "Deriving the RSA Public Key using the corresponding Private Key (rsa):"
openssl rsa -pubout -inform PEM -in "${RSA_PRIVATE_KEY_PEM}" \
    -outform PEM -out "${FOLDER}/rsa_public-key-info_via_private-key_rsa.pem"
openssl rsa -pubout -inform DER -in "${RSA_PRIVATE_KEY_DER}" \
    -outform DER -out "${FOLDER}/rsa_public-key-info_via_private-key_rsa.der"
echo ""

echo "Deriving the RSA Public Key using the corresponding Private Key (rsa):"
openssl pkey -pubout -inform PEM -in "${RSA_PRIVATE_KEY_PEM}" \
    -outform PEM -out "${FOLDER}/rsa_public-key-info_via_private-key_pkey.pem"
openssl pkey -pubout -inform DER -in "${RSA_PRIVATE_KEY_DER}" \
    -outform DER -out "${FOLDER}/rsa_public-key-info_via_private-key_pkey.der"
echo ""

# validate results
echo "RSA Files:"
ls -l -go "${FOLDER}"/rsa_public-key-info_via_private-key*
echo ""

echo "RSA File Types:"
file "${FOLDER}"/rsa_public-key-info_via_private-key*
echo ""

echo "RSA File Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_public-key-info_via_private-key*.der
echo "---"
sha256sum "${FOLDER}"/rsa_public-key-info_via_private-key*.pem

Environment:
  RSA Private Key PEM: 'data/poc/rsa_private-key.pem'
  RSA Private Key DER: 'data/poc/rsa_private-key.der'

Deriving the RSA Public Key using the corresponding Private Key (rsa):


writing RSA key
writing RSA key



Deriving the RSA Public Key using the corresponding Private Key (rsa):

RSA Files:
-rw-r--r--@ 1   294 Sep 20 12:40 data/poc/rsa_public-key-info_via_private-key_pkey.der
-rw-r--r--@ 1   451 Sep 20 12:40 data/poc/rsa_public-key-info_via_private-key_pkey.pem
-rw-r--r--@ 1   294 Sep 20 12:40 data/poc/rsa_public-key-info_via_private-key_rsa.der
-rw-r--r--@ 1   451 Sep 20 12:40 data/poc/rsa_public-key-info_via_private-key_rsa.pem

RSA File Types:
data/poc/rsa_public-key-info_via_private-key_pkey.der: data
data/poc/rsa_public-key-info_via_private-key_pkey.pem: ASCII text
data/poc/rsa_public-key-info_via_private-key_rsa.der:  data
data/poc/rsa_public-key-info_via_private-key_rsa.pem:  ASCII text

RSA File Digests (SHA256):
aba9c431a648c00bbc69fa0df9394bb701e9771413f1d0e86866c29dc4eac72d  data/poc/rsa_public-key-info_via_private-key_pkey.der
aba9c431a648c00bbc69fa0df9394bb701e9771413f1d0e86866c29dc4eac72d  data/poc/rsa_public-key-info_via_private-key_rsa.der
---
a5c2514648e74027fe19088261504d

### Generate RSA Signing Requests

Our goal is to create multiple unique signed public certificates each based on the single private key we just created.  To begin, we must create a set of _certificate signing requests_, each provided with a unique name.  

To uniquely identify each signing request (and later the certificate) we apply a unique ID to the request `CommonName` field of the `Subject`.

In [9]:
%%bash
FOLDER="data/poc"
RSA_PRIVATE_KEY_PEM="${FOLDER}/rsa_private-key.pem"
MAX_SIGNING_REQUESTS=5

echo "Environment:"
echo "  Signing Request Count: ${MAX_SIGNING_REQUESTS}"
echo ""

for ((i=1; i<="${MAX_SIGNING_REQUESTS}"; i++)); do
    GUID=$(uuidgen)
    RSA_SIGNING_REQUEST_PEM="${FOLDER}/rsa_signing-request_${GUID}.pem"
    RSA_SIGNING_REQUEST_DER="${FOLDER}/rsa_signing-request_${GUID}.der"
    
    # creates PEM signing request
    echo "Creating PEM: ${RSA_SIGNING_REQUEST_PEM}"
    openssl req -new -sha256 -inform pem -key "${RSA_PRIVATE_KEY_PEM}" \
      -outform pem -out "${RSA_SIGNING_REQUEST_PEM}" -subj "/O=WARNING\: PRIVATE KEY MADE PUBLIC/CN=${GUID}"

    # converts PEM to DER signing request
    echo "Creating DER: ${RSA_SIGNING_REQUEST_DER}"
    openssl req -inform PEM -in "${RSA_SIGNING_REQUEST_PEM}" \
      -outform DER -out "${RSA_SIGNING_REQUEST_DER}"

done
echo ""

echo "RSA Signing Requests:"
ls -l -go "${FOLDER}"/rsa_signing-request*
echo ""

echo "RSA File Types:"
file "${FOLDER}"/rsa_signing-request*
echo ""

echo "RSA Signing Request Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_signing-request*.der
echo "---"
sha256sum "${FOLDER}"/rsa_signing-request*.pem
echo ""


Environment:
  Signing Request Count: 5

Creating PEM: data/poc/rsa_signing-request_2796EE12-D685-43CE-AB7C-2A8D0313ECBD.pem
Creating DER: data/poc/rsa_signing-request_2796EE12-D685-43CE-AB7C-2A8D0313ECBD.der
Creating PEM: data/poc/rsa_signing-request_5369DFF0-A7E7-476A-8EF8-0829EFD2BF64.pem
Creating DER: data/poc/rsa_signing-request_5369DFF0-A7E7-476A-8EF8-0829EFD2BF64.der
Creating PEM: data/poc/rsa_signing-request_09D0A34B-D935-48DC-AD0F-17FE1D822D00.pem
Creating DER: data/poc/rsa_signing-request_09D0A34B-D935-48DC-AD0F-17FE1D822D00.der
Creating PEM: data/poc/rsa_signing-request_201204F0-9E62-4189-953B-0AB1DD04FF27.pem
Creating DER: data/poc/rsa_signing-request_201204F0-9E62-4189-953B-0AB1DD04FF27.der
Creating PEM: data/poc/rsa_signing-request_2A1C1D8E-13F8-45FB-8E8F-E26E1406BEE4.pem
Creating DER: data/poc/rsa_signing-request_2A1C1D8E-13F8-45FB-8E8F-E26E1406BEE4.der

RSA Signing Requests:
-rw-r--r--@ 1   675 Sep 20 12:43 data/poc/rsa_signing-request_09D0A34B-D935-48DC-AD0F-17FE1D822D

### Self-Sign RSA Signing Requests

Using the single private key, we can self-sign each of the signing requests to create a completed signed certificate for analysis.  For added variability with our certificates, each has a unique validity duration.

In [10]:
%%bash
FOLDER="data/poc"
RSA_PRIVATE_KEY_PEM="${FOLDER}/rsa_private-key.pem"
VALIDITY_DURATION=30
VALIDITY_INCREMENT=10

for RSA_SIGNING_REQUEST_PEM in "${FOLDER}"/rsa_signing-request_*.pem; do
    VALIDITY_DURATION=$((VALIDITY_DURATION+VALIDITY_INCREMENT))
    GUID=$(openssl req -noout -in "${RSA_SIGNING_REQUEST_PEM}" -subject | awk -F 'CN=' '{print $2}')
    RSA_SELF_SIGNED_PUBLIC_KEY_PEM="${FOLDER}/rsa_self-signed-public-key_${GUID}.pem" 
    RSA_SELF_SIGNED_PUBLIC_KEY_DER="${FOLDER}/rsa_self-signed-public-key_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Self-Signed PEM: ${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}"
    openssl x509 -req -sha256 -days ${VALIDITY_DURATION} \
        -keyform PEM -signkey "${RSA_PRIVATE_KEY_PEM}" \
        -inform PEM -in "${RSA_SIGNING_REQUEST_PEM}" \
        -outform PEM -out "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${RSA_SELF_SIGNED_PUBLIC_KEY_DER}"
    openssl x509 -inform PEM -in "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}" \
        -outform DER -out "${RSA_SELF_SIGNED_PUBLIC_KEY_DER}"
    echo ""
done
echo ""

echo "RSA Self-Signed Certificates:"
ls -l -go "${FOLDER}"/rsa_self-signed-public-key*
echo ""

echo "RSA Self-Signed Certificate File Types:"
file "${FOLDER}"/rsa_self-signed-public-key*
echo ""

echo "RSA Self-Signed Certificate Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_self-signed-public-key*.der
echo "---"
sha256sum "${FOLDER}"/rsa_self-signed-public-key*.pem
echo ""


Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_09D0A34B-D935-48DC-AD0F-17FE1D822D00.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=09D0A34B-D935-48DC-AD0F-17FE1D822D00


Converting PEM to DER: data/poc/rsa_self-signed-public-key_09D0A34B-D935-48DC-AD0F-17FE1D822D00.der

Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_201204F0-9E62-4189-953B-0AB1DD04FF27.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=201204F0-9E62-4189-953B-0AB1DD04FF27


Converting PEM to DER: data/poc/rsa_self-signed-public-key_201204F0-9E62-4189-953B-0AB1DD04FF27.der

Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_2796EE12-D685-43CE-AB7C-2A8D0313ECBD.pem


...

Converting PEM to DER: data/poc/rsa_self-signed-public-key_2796EE12-D685-43CE-AB7C-2A8D0313ECBD.der

Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_2A1C1D8E-13F8-45FB-8E8F-E26E1406BEE4.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=2A1C1D8E-13F8-45FB-8E8F-E26E1406BEE4


Converting PEM to DER: data/poc/rsa_self-signed-public-key_2A1C1D8E-13F8-45FB-8E8F-E26E1406BEE4.der

Creating Self-Signed PEM: data/poc/rsa_self-signed-public-key_5369DFF0-A7E7-476A-8EF8-0829EFD2BF64.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=5369DFF0-A7E7-476A-8EF8-0829EFD2BF64


Converting PEM to DER: data/poc/rsa_self-signed-public-key_5369DFF0-A7E7-476A-8EF8-0829EFD2BF64.der


RSA Self-Signed Certificates:
-rw-r--r--@ 1    871 Sep 20 12:44 data/poc/rsa_self-signed-public-key_09D0A34B-D935-48DC-AD0F-17FE1D822D00.der
-rw-r--r--@ 1   1237 Sep 20 12:44 data/poc/rsa_self-signed-public-key_09D0A34B-D935-48DC-AD0F-17FE1D822D00.pem
-rw-r--r--@ 1    871 Sep 20 12:44 data/poc/rsa_self-signed-public-key_201204F0-9E62-4189-953B-0AB1DD04FF27.der
-rw-r--r--@ 1   1237 Sep 20 12:44 data/poc/rsa_self-signed-public-key_201204F0-9E62-4189-953B-0AB1DD04FF27.pem
-rw-r--r--@ 1    871 Sep 20 12:44 data/poc/rsa_self-signed-public-key_2796EE12-D685-43CE-AB7C-2A8D0313ECBD.der
-rw-r--r--@ 1   1237 Sep 20 12:44 data/poc/rsa_self-signed-public-key_2796EE12-D685-43CE-AB7C-2A8D0313ECBD.pem
-rw-r--r--@ 1    871 Sep 20 12:44 data/poc/rsa_self-signed-public-key_2A1C1D8E-13F8-45FB-8E8F-E26E1406BEE4.der
-rw-r--r--@ 1   1237 Sep 20 12:44 data/poc/rsa_self-signed-public-key_2A1C1D8E-13F8-45FB-8E

### Display Certificate Details (Briefly)

Confirming our certificates have unique names and validity periods.

In [11]:
%%bash
FOLDER="data/poc"

# from signed certificates
echo "Basic Certificate Details:"
for RSA_SELF_SIGNED_PUBLIC_KEY_PEM in "${FOLDER}"/rsa_self-signed-public-key*.pem; do
    echo "--------------------------------------------"
    openssl x509 -noout -in "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}" \
        -subject -serial -startdate -enddate
done
echo ""

Basic Certificate Details:
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=09D0A34B-D935-48DC-AD0F-17FE1D822D00
serial=42B721D65893A0A0746727BBF613FBB74D549388
notBefore=Sep 20 16:44:22 2024 GMT
notAfter=Oct 30 16:44:22 2024 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=201204F0-9E62-4189-953B-0AB1DD04FF27
serial=7B33A3A4996E8C9A107DE95C4DFE1214E59B0ECB
notBefore=Sep 20 16:44:22 2024 GMT
notAfter=Nov  9 16:44:22 2024 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=2796EE12-D685-43CE-AB7C-2A8D0313ECBD
serial=5B035CD395BDBE114EBAF181F934425CFE8D25C2
notBefore=Sep 20 16:44:22 2024 GMT
notAfter=Nov 19 16:44:22 2024 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=2A1C1D8E-13F8-45FB-8E8F-E26E1406BEE4
serial=55F7726B9865830DE41CC45BA1CB336042AD9E4D
notBefore=Sep 20 16:44:22 2024 GMT
notAfter=Nov 29 16:44:2

### Extract RSA SubjectPublicKeyInfo

We already derived the raw public key material earlier by accessing the private key for this information.  Here, we'll shift our focus to both the signing requests and self-signed certificates for analysis.

First, we'll access the signing requests to extract the `SubjectPublicKeyInfo` data and convert it to DER (binary) format.  Then we'll access each self-signed certificate to perform a similar action.

> Note: when using the `-pubkey` option, the output will often contain two sections, in ASCII format, defining the `PUBLIC KEY` and `CERTIFICATE REQUEST`.  We use `sed` to parse out only the `PUBLIC KEY` portion of this output.

In [12]:
%%bash
FOLDER="data/poc"

echo "Extracting the SubjectPublicKeyInfo from RSA Signing Requests:"
echo "--------------------------------------------"
# from signing request
for RSA_SIGNING_REQUEST_PEM in "${FOLDER}"/rsa_signing-request*.pem; do
    GUID=$(openssl req -noout -in "${RSA_SIGNING_REQUEST_PEM}" -subject | awk -F 'CN=' '{print $2}')
    RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM="${FOLDER}/rsa_public-key-info_via_signing-request_${GUID}.pem" 
    RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER="${FOLDER}/rsa_public-key-info_via_signing-request_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Public Key Info PEM: ${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}"
    openssl req -pubkey -inform PEM -in "${RSA_SIGNING_REQUEST_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER}"
    openssl asn1parse -inform PEM -in "${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}" -out "${RSA_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER}"
    echo ""
done
echo ""

echo "Extracting the SubjectPublicKeyInfo from RSA Self-Signed Certificates:"
echo "--------------------------------------------"
# from self-signed public key
for RSA_SELF_SIGNED_PUBLIC_KEY_PEM in "${FOLDER}"/rsa_self-signed-public-key*.pem; do
    GUID=$(openssl x509 -noout -in "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}" -subject | awk -F 'CN=' '{print $2}')
    RSA_PUBLIC_KEY_INFO_SELF_SIGNED_PEM="${FOLDER}/rsa_public-key-info_via_self-signed_${GUID}.pem" 
    RSA_PUBLIC_KEY_INFO_SELF_SIGNED_DER="${FOLDER}/rsa_public-key-info_via_self-signed_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Public Key Info PEM: ${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}"
    openssl x509 -pubkey -inform PEM -in "${RSA_SELF_SIGNED_PUBLIC_KEY_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_DER}"
    openssl asn1parse -inform PEM -in "${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}" -out "${RSA_PUBLIC_KEY_INFO_SELF_SIGNED_DER}"
    echo ""
done
echo ""


Extracting the SubjectPublicKeyInfo from RSA Signing Requests:
--------------------------------------------
Creating Public Key Info PEM: data/poc/rsa_public-key-info_via_signing-request_09D0A34B-D935-48DC-AD0F-17FE1D822D00.pem
Converting PEM to DER: data/poc/rsa_public-key-info_via_signing-request_09D0A34B-D935-48DC-AD0F-17FE1D822D00.der
    0:d=0  hl=4 l= 290 cons: SEQUENCE          
    4:d=1  hl=2 l=  13 cons: SEQUENCE          
    6:d=2  hl=2 l=   9 prim: OBJECT            :rsaEncryption
   17:d=2  hl=2 l=   0 prim: NULL              
   19:d=1  hl=4 l= 271 prim: BIT STRING        

Creating Public Key Info PEM: data/poc/rsa_public-key-info_via_signing-request_201204F0-9E62-4189-953B-0AB1DD04FF27.pem
Converting PEM to DER: data/poc/rsa_public-key-info_via_signing-request_201204F0-9E62-4189-953B-0AB1DD04FF27.der
    0:d=0  hl=4 l= 290 cons: SEQUENCE          
    4:d=1  hl=2 l=  13 cons: SEQUENCE          
    6:d=2  hl=2 l=   9 prim: OBJECT            :rsaEncryption
   17:d=2  hl

### Compare PublicKeyInfo Digests

In summary, we've performed the following actions:
1. Derived the public key from a single private key 
<br>(see `*public-key-info_via_private_key*.der`).
2. Extracted the public key material from multiple certificate signing requests
<br>(see `*public-key-info_via_signing-request*.der`).
3. Extracted the public key material from multiple self-signed certificates 
<br>(see `*public-key-info_via_self-signed*.der`).

Do we have message digests which match across all these different exports?

> Note 1: If you do not see the results you expect, make sure you are [Erase All RSA Key Data](#Erase-All-RSA-Key-Data) between test runs.

If you want more samples, then follow these steps without erasing or resetting the RSA data.
1. Run [Generate RSA Signing Requests](#generate-rsa-signing-requests) multiple times.  e.g. 4 more times for a total of 25 signing requests.
2. Run [Self-Sign RSA Signing Requests](#self-sign-rsa-signing-requests) just once.  
3. Run [Extract RSA SubjectPublicKeyInfo](#extract-rsa-subjectpublickeyinfo) just once.
4. Run this cell again.

In [13]:
%%bash
FOLDER="data/poc"

# basic error checking
if ! find "${FOLDER}" -type f -name "rsa_self-signed-public-key*.der" | grep -q .; then
    echo "WARNING: No RSA Self-Signed Certificates found."
    echo "         Please run the previous cells to generate RSA content."
    exit
fi

UNIQUE_DER_DIGEST_COUNT=$(sha256sum "${FOLDER}"/rsa_self-signed-public-key*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

UNIQUE_SPKI_DIGEST_COUNT=$(sha256sum "${FOLDER}"/rsa_public-key-info*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

# if UNIQUE_SPKI_DIGEST_COUNT greater than 1 then issue a warning
if [ "${UNIQUE_SPKI_DIGEST_COUNT}" -gt 1 ]; then
    echo "WARNING: Multiple SubjectPublicKeyInfo Digests found."
    echo "         Did you forget to erase all key data between runs?"
    echo ""
fi

echo "Summary of Unique RSA Key Data:"
echo "   Signed Certificates: ${UNIQUE_DER_DIGEST_COUNT}"
echo "  SubjectPublicKeyInfo: ${UNIQUE_SPKI_DIGEST_COUNT}"
echo ""

echo "Certificate (DER) Digests (SHA256):"
sha256sum "${FOLDER}"/rsa_self-signed-public-key*.der
echo ""

echo "SubjectPublicKeyInfo Digests (SHA256):"
echo "[SubjectPublicKeyInfo derived from the Private Key]"
sha256sum "${FOLDER}"/rsa_public-key-info_via_private-key*.der | sort
echo "[SubjectPublicKeyInfo extracted from Signing Requests]"
sha256sum "${FOLDER}"/rsa_public-key-info_via_signing-request*.der | sort
echo "[SubjectPublicKeyInfo extracted from Self-Signed Certificates]"
sha256sum "${FOLDER}"/rsa_public-key-info_via_self-signed*.der | sort


Summary of Unique RSA Key Data:
   Signed Certificates: 5
  SubjectPublicKeyInfo: 1

Certificate (DER) Digests (SHA256):
df01634a658e95bb17a250e46335b2f436c0b87522c6d414becf0510d3da00c2  data/poc/rsa_self-signed-public-key_09D0A34B-D935-48DC-AD0F-17FE1D822D00.der
ac8204fcf6fabd40b0c0da03991ce776ca11d2c76e7be4d833ed6a1da899d1c2  data/poc/rsa_self-signed-public-key_201204F0-9E62-4189-953B-0AB1DD04FF27.der
cd32666f712a104be1cde50fcc109b0a87e651355fb1110273798f7c88f85538  data/poc/rsa_self-signed-public-key_2796EE12-D685-43CE-AB7C-2A8D0313ECBD.der
894a054992b34eeffc7f7f31644299bb143b4bc73b9a0bad6c747d87fedd0d0c  data/poc/rsa_self-signed-public-key_2A1C1D8E-13F8-45FB-8E8F-E26E1406BEE4.der
2e5600877faab408bd02de5bb8efdb02235c16112b68af3d5957cd4d2dd2119a  data/poc/rsa_self-signed-public-key_5369DFF0-A7E7-476A-8EF8-0829EFD2BF64.der

SubjectPublicKeyInfo Digests (SHA256):
[SubjectPublicKeyInfo derived from the Private Key]
aba9c431a648c00bbc69fa0df9394bb701e9771413f1d0e86866c29dc4eac72d  data/p

### Erase All RSA Key Data

Use this to reset the proof-of-concept data related to RSA key data.

In [1]:
%%bash
FOLDER="data/poc"

# if folder exists, remove it
rm -rf "${FOLDER}"/rsa_*

## EC

Resources:

1. [openssl-asn1parse](https://docs.openssl.org/master/man1/openssl-asn1parse/) processing ASN.1 files
2. [openssl-ec](https://docs.openssl.org/master/man1/openssl-ec/) generating elliptic curve private keys
3. [openssl-pkey](https://docs.openssl.org/master/man1/openssl-pkey/) interacting with private keys
4. [openssl-req](https://docs.openssl.org/master/man1/openssl-req/) processing signing-requests
5. [openssl-x509](https://docs.openssl.org/master/man1/openssl-x509/) processing X.509 certificates


### Generate EC Private Key

For illustration purposes, let us create a single RSA private key. 

> Warning: Please do not use these keys in any production or live environment as they are intended for demonstration purposes only.  As a _demo_, you have no control over who may have viewed or obtain a copy of this key material making these private keys potentially _public_.

In [14]:
%%bash
# preamble
FOLDER="data/poc"
mkdir -p "${FOLDER}"
EC_PRIVATE_KEY_PEM="${FOLDER}/ec_private-key.pem"
EC_PRIVATE_KEY_DER="${FOLDER}/ec_private-key.der"
EC_CURVE="prime256v1" # `prime256v1` and `secp384r1`

# validate environment
echo "Environment:"
echo "  Working Folder:  '${FOLDER}'"
echo "  Private Key PEM: '${EC_PRIVATE_KEY_PEM}'"
echo "  Private Key DER: '${EC_PRIVATE_KEY_DER}'"
echo "  Elliptic Curve:  '${EC_CURVE}'"
echo ""

# execute
echo "Generating EC Private Key:"

# create the private key in PEM form.
openssl ecparam -genkey -name "${EC_CURVE}" -noout -out "${EC_PRIVATE_KEY_PEM}"

# convert private key from PEM (ASCII) to DER (binary)
openssl ec -inform pem -in "${EC_PRIVATE_KEY_PEM}" -outform der -out "${EC_PRIVATE_KEY_DER}"
echo ""

# valdiate result
echo "Files:"
ls -l -go "${FOLDER}"/ec_private-key.*
echo ""

echo "File Types:"
file "${FOLDER}"/ec_private-key.*
echo ""

echo "File Digests (SHA256):"
sha256sum "${FOLDER}"/ec_private-key.*


Environment:
  Working Folder:  'data/poc'
  Private Key PEM: 'data/poc/ec_private-key.pem'
  Private Key DER: 'data/poc/ec_private-key.der'
  Elliptic Curve:  'prime256v1'

Generating EC Private Key:


read EC key
writing EC key



Files:
-rw-------@ 1   121 Sep 20 12:49 data/poc/ec_private-key.der
-rw-------@ 1   227 Sep 20 12:49 data/poc/ec_private-key.pem

File Types:
data/poc/ec_private-key.der: data
data/poc/ec_private-key.pem: PEM EC private key

File Digests (SHA256):
6ecd9efe2772216b19baf6d6f264ab8d125b00a8d25b397305afe6d448483763  data/poc/ec_private-key.der
341666e4ecfc81ec67f107395718431c369ec39a6a1f514d4fc9ef6579e82605  data/poc/ec_private-key.pem


### Derive the EC Public Key

The raw private key material itself does not contain the public key.  However we can derive the cryptographic public key material from the private key using one of the `openssl` command options.  
1. `openssl ec -pubout ...`
2. `openssl pkey -pubout ...`

In [15]:
%%bash
# preamble
FOLDER="data/poc"
EC_PRIVATE_KEY_PEM="${FOLDER}/ec_private-key.pem"
EC_PRIVATE_KEY_DER="${FOLDER}/ec_private-key.der"

# validate environment
echo "Environment:"
echo "  EC Private Key PEM: '${EC_PRIVATE_KEY_PEM}'"
echo "  EC Private Key DER: '${EC_PRIVATE_KEY_DER}'"
echo ""

# execute
echo "Deriving the EC Public Key using the corresponding Private Key (ec):"
openssl ec -pubout -inform PEM -in "${EC_PRIVATE_KEY_PEM}" \
    -outform PEM -out "${FOLDER}/ec_public-key-info_via_private-key_ec.pem"
openssl ec -pubout -inform DER -in "${EC_PRIVATE_KEY_DER}" \
    -outform DER -out "${FOLDER}/ec_public-key-info_via_private-key_ec.der"
echo ""

echo "Deriving the EC Public Key using the corresponding Private Key (pkey):"
openssl pkey -pubout -inform PEM -in "${EC_PRIVATE_KEY_PEM}" \
    -outform PEM -out "${FOLDER}"/ec_public-key-info_via_private-key_pkey.pem
openssl pkey -pubout -inform DER -in "${EC_PRIVATE_KEY_DER}" \
    -outform DER -out "${FOLDER}"/ec_public-key-info_via_private-key_pkey.der
echo ""

# validate results
echo "EC Files:"
ls -l -go "${FOLDER}"/ec_public-key-info_via_private-key*
echo ""

echo "EC File Types:"
file "${FOLDER}"/ec_public-key-info_via_private-key*
echo ""

echo "EC File Digests (SHA256):"
sha256sum "${FOLDER}"/ec_public-key-info_via_private-key*.der
echo "---"
sha256sum "${FOLDER}"/ec_public-key-info_via_private-key*.pem

Environment:
  EC Private Key PEM: 'data/poc/ec_private-key.pem'
  EC Private Key DER: 'data/poc/ec_private-key.der'

Deriving the EC Public Key using the corresponding Private Key (ec):


read EC key
writing EC key
read EC key
writing EC key



Deriving the EC Public Key using the corresponding Private Key (pkey):

EC Files:
-rw-r--r--@ 1    91 Sep 20 12:49 data/poc/ec_public-key-info_via_private-key_ec.der
-rw-r--r--@ 1   178 Sep 20 12:49 data/poc/ec_public-key-info_via_private-key_ec.pem
-rw-r--r--@ 1    91 Sep 20 12:49 data/poc/ec_public-key-info_via_private-key_pkey.der
-rw-r--r--@ 1   178 Sep 20 12:49 data/poc/ec_public-key-info_via_private-key_pkey.pem

EC File Types:
data/poc/ec_public-key-info_via_private-key_ec.der:   data
data/poc/ec_public-key-info_via_private-key_ec.pem:   ASCII text
data/poc/ec_public-key-info_via_private-key_pkey.der: data
data/poc/ec_public-key-info_via_private-key_pkey.pem: ASCII text

EC File Digests (SHA256):
1d292188d76d595eda2a8a3bddd05fec5e197f59bbe50abc4ae7954746862b37  data/poc/ec_public-key-info_via_private-key_ec.der
1d292188d76d595eda2a8a3bddd05fec5e197f59bbe50abc4ae7954746862b37  data/poc/ec_public-key-info_via_private-key_pkey.der
---
5c8f73adeb13f97b8a2d7ccf33bd9ca1ac7dbe72377dc8

### Generate EC Signing Requests

Our goal is to create multiple unique signed public certificates each based on the single private key we just created.  To begin, we must create a set of _certificate signing requests_, each provided with a unique name.

To uniquely identify each signing request (and later the certificate) we apply a unique ID to the request `CommonName` field of the `Subject`.

In [16]:
%%bash
# preamble
FOLDER="data/poc"
EC_PRIVATE_KEY_PEM="${FOLDER}/ec_private-key.pem"
MAX_SIGNING_REQUESTS=5


echo "Environment:"
echo "  Signing Request Count: ${MAX_SIGNING_REQUESTS}"
echo ""

for ((i=1; i<="${MAX_SIGNING_REQUESTS}"; i++)); do
    GUID=$(uuidgen)
    EC_SIGNING_REQUEST_PEM="${FOLDER}/ec_signing-request_${GUID}.pem"
    EC_SIGNING_REQUEST_DER="${FOLDER}/ec_signing-request_${GUID}.der"
    
    # creates PEM signing request
    echo "Creating PEM: ${EC_SIGNING_REQUEST_PEM}"
    openssl req -new -sha256 -inform pem -key "${EC_PRIVATE_KEY_PEM}" \
      -outform pem -out "${EC_SIGNING_REQUEST_PEM}" -subj "/O=WARNING\: PRIVATE KEY MADE PUBLIC/CN=${GUID}"

    # converts PEM to DER signing request
    echo "Creating DER: ${EC_SIGNING_REQUEST_DER}"
    openssl req -inform PEM -in "${EC_SIGNING_REQUEST_PEM}" \
      -outform DER -out "${EC_SIGNING_REQUEST_DER}"

done
echo ""

echo "EC Signing Requests:"
ls -l -go "${FOLDER}"/ec_signing-request*
echo ""

echo "EC File Types:"
file "${FOLDER}"/ec_signing-request*
echo ""

echo "EC Signing Request Digests (SHA256):"
sha256sum "${FOLDER}"/ec_signing-request*.der
echo "---"
sha256sum "${FOLDER}"/ec_signing-request*.pem
echo ""


Environment:
  Signing Request Count: 5



Creating PEM: data/poc/ec_signing-request_A4080B2A-CA07-4631-AE44-58EBD029A171.pem
Creating DER: data/poc/ec_signing-request_A4080B2A-CA07-4631-AE44-58EBD029A171.der
Creating PEM: data/poc/ec_signing-request_46159700-E905-4062-993F-7F58175F09AA.pem
Creating DER: data/poc/ec_signing-request_46159700-E905-4062-993F-7F58175F09AA.der
Creating PEM: data/poc/ec_signing-request_D97D6D32-AD8F-4CE7-92BB-6F822841EA76.pem
Creating DER: data/poc/ec_signing-request_D97D6D32-AD8F-4CE7-92BB-6F822841EA76.der
Creating PEM: data/poc/ec_signing-request_F36C6183-1CCA-4E4B-99FD-FABBE612B927.pem
Creating DER: data/poc/ec_signing-request_F36C6183-1CCA-4E4B-99FD-FABBE612B927.der
Creating PEM: data/poc/ec_signing-request_F6930B2C-608D-412E-B5EA-7CB961C71657.pem
Creating DER: data/poc/ec_signing-request_F6930B2C-608D-412E-B5EA-7CB961C71657.der

EC Signing Requests:
-rw-r--r--@ 1   280 Sep 20 12:49 data/poc/ec_signing-request_46159700-E905-4062-993F-7F58175F09AA.der
-rw-r--r--@ 1   452 Sep 20 12:49 data/poc/ec_s

### Self-Sign EC Signing Requests

Using the single private key, we can self-sign each of the signing requests to create a completed signed certificate for analysis.

In [17]:
%%bash
# preamble
FOLDER="data/poc"
EC_PRIVATE_KEY_PEM="${FOLDER}/ec_private-key.pem"
VALIDITY_DURATION=30
VALIDITY_INCREMENT=10

for EC_SIGNING_REQUEST_PEM in "${FOLDER}"/ec_signing-request*.pem; do
    VALIDITY_DURATION=$((VALIDITY_DURATION+VALIDITY_INCREMENT))
    GUID=$(openssl req -noout -in "${EC_SIGNING_REQUEST_PEM}" -subject | awk -F 'CN=' '{print $2}')
    EC_SELF_SIGNED_PUBLIC_KEY_PEM="${FOLDER}/ec_self-signed-public-key_${GUID}.pem" 
    EC_SELF_SIGNED_PUBLIC_KEY_DER="${FOLDER}/ec_self-signed-public-key_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Self-Signed PEM: ${EC_SELF_SIGNED_PUBLIC_KEY_PEM}"
    openssl x509 -req -sha256 -days ${VALIDITY_DURATION} \
        -keyform PEM -signkey "${EC_PRIVATE_KEY_PEM}" \
        -inform PEM -in "${EC_SIGNING_REQUEST_PEM}" \
        -outform PEM -out "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${EC_SELF_SIGNED_PUBLIC_KEY_DER}"
    openssl x509 -inform PEM -in "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}" \
        -outform DER -out "${EC_SELF_SIGNED_PUBLIC_KEY_DER}"
done
echo ""

echo "EC Self-Signed Certificates:"
ls -l -go "${FOLDER}"/ec_self-signed-public-key*
echo ""

echo "EC Self-Signed Certificate File Types:"
file "${FOLDER}"/ec_self-signed-public-key*
echo ""

echo "EC Self-Signed Certificate SHA256 Digests:"
sha256sum "${FOLDER}"/ec_self-signed-public-key*.der
echo "---"
sha256sum "${FOLDER}"/ec_self-signed-public-key*.pem
echo ""


Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_46159700-E905-4062-993F-7F58175F09AA.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=46159700-E905-4062-993F-7F58175F09AA


Converting PEM to DER: data/poc/ec_self-signed-public-key_46159700-E905-4062-993F-7F58175F09AA.der
Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_A4080B2A-CA07-4631-AE44-58EBD029A171.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=A4080B2A-CA07-4631-AE44-58EBD029A171


Converting PEM to DER: data/poc/ec_self-signed-public-key_A4080B2A-CA07-4631-AE44-58EBD029A171.der
Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_D97D6D32-AD8F-4CE7-92BB-6F822841EA76.pem


...

Converting PEM to DER: data/poc/ec_self-signed-public-key_D97D6D32-AD8F-4CE7-92BB-6F822841EA76.der
Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_F36C6183-1CCA-4E4B-99FD-FABBE612B927.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=F36C6183-1CCA-4E4B-99FD-FABBE612B927


Converting PEM to DER: data/poc/ec_self-signed-public-key_F36C6183-1CCA-4E4B-99FD-FABBE612B927.der
Creating Self-Signed PEM: data/poc/ec_self-signed-public-key_F6930B2C-608D-412E-B5EA-7CB961C71657.pem


Certificate request self-signature ok
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=F6930B2C-608D-412E-B5EA-7CB961C71657


Converting PEM to DER: data/poc/ec_self-signed-public-key_F6930B2C-608D-412E-B5EA-7CB961C71657.der

EC Self-Signed Certificates:
-rw-r--r--@ 1   474 Sep 20 12:49 data/poc/ec_self-signed-public-key_46159700-E905-4062-993F-7F58175F09AA.der
-rw-r--r--@ 1   696 Sep 20 12:49 data/poc/ec_self-signed-public-key_46159700-E905-4062-993F-7F58175F09AA.pem
-rw-r--r--@ 1   475 Sep 20 12:49 data/poc/ec_self-signed-public-key_A4080B2A-CA07-4631-AE44-58EBD029A171.der
-rw-r--r--@ 1   700 Sep 20 12:49 data/poc/ec_self-signed-public-key_A4080B2A-CA07-4631-AE44-58EBD029A171.pem
-rw-r--r--@ 1   476 Sep 20 12:49 data/poc/ec_self-signed-public-key_D97D6D32-AD8F-4CE7-92BB-6F822841EA76.der
-rw-r--r--@ 1   700 Sep 20 12:49 data/poc/ec_self-signed-public-key_D97D6D32-AD8F-4CE7-92BB-6F822841EA76.pem
-rw-r--r--@ 1   475 Sep 20 12:49 data/poc/ec_self-signed-public-key_F36C6183-1CCA-4E4B-99FD-FABBE612B927.der
-rw-r--r--@ 1   700 Sep 20 12:49 data/poc/ec_self-signed-public-key_F36C6183-1CCA-4E4B-99FD-FABBE612B927.pem

### Display Certificate Details (Briefly)

Confirming our certificates have unique names and validity periods.

In [18]:
%%bash
FOLDER="data/poc"

# from signed certificates
echo "Basic Certificate Details:"
for EC_SELF_SIGNED_PUBLIC_KEY_PEM in "${FOLDER}"/ec_self-signed-public-key*.pem; do
    echo "--------------------------------------------"
    openssl x509 -noout -in "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}" \
        -subject -serial -startdate -enddate
done
echo ""

Basic Certificate Details:
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=46159700-E905-4062-993F-7F58175F09AA
serial=1F695E05329667E9CE7F405FFB31C0809356FB1E
notBefore=Sep 20 16:49:21 2024 GMT
notAfter=Oct 30 16:49:21 2024 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=A4080B2A-CA07-4631-AE44-58EBD029A171
serial=6D2487B186C9AC7E446684ABCD56C7EE7036378B
notBefore=Sep 20 16:49:21 2024 GMT
notAfter=Nov  9 16:49:21 2024 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=D97D6D32-AD8F-4CE7-92BB-6F822841EA76
serial=20E717B7975D58CCD4DCB9AAC26440E473D536D2
notBefore=Sep 20 16:49:21 2024 GMT
notAfter=Nov 19 16:49:21 2024 GMT
--------------------------------------------
subject=O=WARNING: PRIVATE KEY MADE PUBLIC, CN=F36C6183-1CCA-4E4B-99FD-FABBE612B927
serial=1BC307FAFBD14ACCFFF35F7EBD1E5A4EAE3CF9D0
notBefore=Sep 20 16:49:21 2024 GMT
notAfter=Nov 29 16:49:2

### Extract EC SubjectPublicKeyInfo

We already derived the raw public key material earlier by accessing the private key for this information.  Here, we'll shift our focus to both the signing requests and self-signed certificates for analysis.

First, we'll access the signing requests to extract the `SubjectPublicKeyInfo` data and convert it to DER (binary) format.  Then we'll access each self-signed certificate to perform a similar action.

> Note: when using the `-pubkey` option, the output will often contain two sections, in ASCII format, defining the `PUBLIC KEY` and `CERTIFICATE REQUEST`.  We use `sed` to parse out only the `PUBLIC KEY` portion of this output.

In [19]:
%%bash
FOLDER="data/poc"

echo "Extracting the SubjectPublicKeyInfo from EC Signing Requests:"
echo "--------------------------------------------"
# from signing request
for EC_SIGNING_REQUEST_PEM in "${FOLDER}"/ec_signing-request*.pem; do
    GUID=$(openssl req -noout -in "${EC_SIGNING_REQUEST_PEM}" -subject | awk -F 'CN=' '{print $2}')
    EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM="${FOLDER}/ec_public-key-info_via_signing-request_${GUID}.pem" 
    EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER="${FOLDER}/ec_public-key-info_via_signing-request_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Public Key Info PEM: ${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}"
    openssl req -pubkey -inform PEM -in "${EC_SIGNING_REQUEST_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER}"
    openssl asn1parse -inform PEM -in "${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_PEM}" \
        -out "${EC_PUBLIC_KEY_INFO_SIGNING_REQUEST_DER}"
done
echo ""

echo "Extracting the SubjectPublicKeyInfo from EC Self-Signed Certificates:"
echo "--------------------------------------------"
# from self-signed public key
for EC_SELF_SIGNED_PUBLIC_KEY_PEM in "${FOLDER}"/ec_self-signed-public-key*.pem; do
    echo "Self-Signed Key: ${EC_SELF_SIGNED_PUBLIC_KEY_PEM}"
    GUID=$(openssl x509 -noout -in "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}" -subject | awk -F 'CN=' '{print $2}')
    EC_PUBLIC_KEY_INFO_SELF_SIGNED_PEM="${FOLDER}/ec_public-key-info_via_self-signed_${GUID}.pem" 
    EC_PUBLIC_KEY_INFO_SELF_SIGNED_DER="${FOLDER}/ec_public-key-info_via_self-signed_${GUID}.der"  

    # creating PEM signed certificate
    echo "Creating Public Key Info PEM: ${EC_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}"
    openssl x509 -pubkey -inform PEM -in "${EC_SELF_SIGNED_PUBLIC_KEY_PEM}" | \
        sed -n '/BEGIN\ PUBLIC\ KEY/,/END\ PUBLIC\ KEY/p' > "${EC_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}"

    # converting PEM to DER
    echo "Converting PEM to DER: ${EC_PUBLIC_KEY_INFO_SELF_SIGNED_DER}"
    openssl asn1parse -inform PEM -in "${EC_PUBLIC_KEY_INFO_SELF_SIGNED_PEM}" \
        -out "${EC_PUBLIC_KEY_INFO_SELF_SIGNED_DER}"
done
echo ""

Extracting the SubjectPublicKeyInfo from EC Signing Requests:
--------------------------------------------
Creating Public Key Info PEM: data/poc/ec_public-key-info_via_signing-request_46159700-E905-4062-993F-7F58175F09AA.pem
Converting PEM to DER: data/poc/ec_public-key-info_via_signing-request_46159700-E905-4062-993F-7F58175F09AA.der
    0:d=0  hl=2 l=  89 cons: SEQUENCE          
    2:d=1  hl=2 l=  19 cons: SEQUENCE          
    4:d=2  hl=2 l=   7 prim: OBJECT            :id-ecPublicKey
   13:d=2  hl=2 l=   8 prim: OBJECT            :prime256v1
   23:d=1  hl=2 l=  66 prim: BIT STRING        
Creating Public Key Info PEM: data/poc/ec_public-key-info_via_signing-request_A4080B2A-CA07-4631-AE44-58EBD029A171.pem
Converting PEM to DER: data/poc/ec_public-key-info_via_signing-request_A4080B2A-CA07-4631-AE44-58EBD029A171.der
    0:d=0  hl=2 l=  89 cons: SEQUENCE          
    2:d=1  hl=2 l=  19 cons: SEQUENCE          
    4:d=2  hl=2 l=   7 prim: OBJECT            :id-ecPublicKey
   13:

### Compare PublicKeyInfo Digests

In summary, we've performed the following actions:
1. Derived the public key from a single private key 
<br>(see `ec_public-key-info_via_private_key*.der`).
2. Extracted the public key material from multiple certificate signing requests
<br>(see `ec_public-key-info_via_signing-request*.der`).
3. Extracted the public key material from multiple self-signed certificates 
<br>(see `ec_public-key-info_via_self-signed*.der`).

Do we have message digests which match across all these different exports?

> Note: If you do not see the results you expect, make sure you are [Erase All EC Key Data](#erase-all-ec-key-data)) between test runs.

If you want more samples, then follow these steps without erasing or resetting the EC data.
1. Run [Generate EC Signing Requests](#generate-ec-signing-requests) multiple times.  e.g. 4 more times for a total of 25 signing requests.
2. Run [Self-Sign EC Signing Requests](#self-sign-ec-signing-requests) just once.  
3. Run [Extract EC SubjectPublicKeyInfo](#extract-ec-subjectpublickeyinfo) just once.
4. Run this cell again.

In [20]:
%%bash
FOLDER="data/poc"

# basic error checking
if ! find "${FOLDER}" -type f -name "ec_self-signed-public-key*.der" | grep -q .; then
    echo "WARNING: No EC Self-Signed Certificates found."
    echo "         Please run the previous cells to generate EC content."
    exit
fi

UNIQUE_DER_DIGEST_COUNT=$(sha256sum "${FOLDER}"/ec_self-signed-public-key*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

UNIQUE_SPKI_DIGEST_COUNT=$(sha256sum "${FOLDER}"/ec_public-key-info*.der \
    | awk '{print $1}' | sort | uniq | wc -l | awk '{print $1}')

if [ "${UNIQUE_SPKI_DIGEST_COUNT}" -gt 1 ]; then
    echo "WARNING: Multiple SubjectPublicKeyInfo Digests found."
    echo "         Did you forget to erase all key data between runs?"
    echo ""
fi

echo "Summary of Unique CC Key Data:"
echo "   Signed Certificates: ${UNIQUE_DER_DIGEST_COUNT}"
echo "  SubjectPublicKeyInfo: ${UNIQUE_SPKI_DIGEST_COUNT}"
echo ""

echo "Certificate (DER) Digests (SHA256):"
sha256sum "${FOLDER}"/ec_self-signed-public-key*.der
echo ""

echo "SubjectPublicKeyInfo Digests (SHA256):"
echo "[SubjectPublicKeyInfo derived from the Private Key]"
sha256sum "${FOLDER}"/ec_public-key-info_via_private-key*.der | sort
echo "[SubjectPublicKeyInfo extracted from Signing Requests]"
sha256sum "${FOLDER}"/ec_public-key-info_via_signing-request*.der | sort
echo "[SubjectPublicKeyInfo extracted from Self-Signed Certificates]"
sha256sum "${FOLDER}"/ec_public-key-info_via_self-signed*.der | sort

Summary of Unique CC Key Data:
   Signed Certificates: 5
  SubjectPublicKeyInfo: 1

Certificate (DER) Digests (SHA256):
c8018bad234c426406d41d6b26ab40898b07fd56d2c7974ede6a49ed5eb47e70  data/poc/ec_self-signed-public-key_46159700-E905-4062-993F-7F58175F09AA.der
b99e83f6578aa8e65e246734f092ad6545704a2ee021a85d57181ad6201504b2  data/poc/ec_self-signed-public-key_A4080B2A-CA07-4631-AE44-58EBD029A171.der
0477a2a5aa083c562b08f4739ab585fc66e9bd95508bd76f09c1bcef51673f2c  data/poc/ec_self-signed-public-key_D97D6D32-AD8F-4CE7-92BB-6F822841EA76.der
193fd954ba167c09139ea60d6124a0826d6f5b8dca2539fb70454efae7d32b44  data/poc/ec_self-signed-public-key_F36C6183-1CCA-4E4B-99FD-FABBE612B927.der
e4cfab72093bd3a9a387313e836e12bbf032901594a4eac996cd1381d78de515  data/poc/ec_self-signed-public-key_F6930B2C-608D-412E-B5EA-7CB961C71657.der

SubjectPublicKeyInfo Digests (SHA256):
[SubjectPublicKeyInfo derived from the Private Key]
1d292188d76d595eda2a8a3bddd05fec5e197f59bbe50abc4ae7954746862b37  data/poc/ec_

### Erase All EC Key Data

Use this to reset the proof-of-concept data related to EC key data.

In [2]:
%%bash
FOLDER="data/poc"

# if folder exists, remove it
rm -rf "${FOLDER}"/ec_*

# Additional OpenSSL Commands and Data


## Cipher Suites

How to check cipher suite support, which should correlate with certificate type (`RSA` vs `ECDSA`).  List of available cipher suites from OpenSSL or OpenSSL compatible (e.g. LibreSSL).   Note, these are based on _your_ local OpenSSL installation.

In [ ]:
%%bash
openssl ciphers -v

## OpenSSL Cipher Suites to JSON

In [ ]:
import json

def openssl_cipher_suites_to_json(openssl_result:list) -> list:
    """Convers OpenSSL verbose cipher suite output to JSON.
    
    cipher_suites = [
        {
            "Cipher Suite": str,
            "Protocol": str,
            "Key Exchange": str,
            "Authentication": str,
            "Encryption": str,
            "MAC": str
        },
        {...}
    ]

    Args:
        openssl_result (list): list of lines from 'openssl ciphers -v'

    Returns:
        list: list of dictionaries
    """
    cipher_suites = []
    for line in openssl_result:
        fields = line.split()
        cipher_suite = {}
        cipher_suite['Cipher Suite'] = fields[0]
        cipher_suite['Protocol'] = fields[1]
        cipher_suite['Key Exchange'] = fields[2].split('=')[1]
        cipher_suite['Authentication'] = fields[3].split('=')[1]
        cipher_suite['Encryption'] = fields[4].split('=')[1]
        cipher_suite['MAC'] = fields[5].split('=')[1]

        cipher_suites.append(cipher_suite)
    return cipher_suites

# using a Jupyter 'feature'.  Caution executing external code or binaries from within your notebooks.
openssl_result = !openssl ciphers -v # use 'RSA' or 'AES' or other filter keywords

suites_json = openssl_cipher_suites_to_json(openssl_result)

print(json.dumps(suites_json, indent=2))


OpenSSL RSA Cipher Suites

We can query `openssl` for various cipher suites based on one or more cipher string values.  See [OpenSSL man page on Ciphers: Cipher Strings](https://docs.openssl.org/1.1.1/man1/ciphers/)

In [ ]:
%%bash

# create an array of strings "RSA", "ECDH", "DHE", "ECDSA", "AES", "RSA"
CIPHERS=("ECDH" "DHE" "ECDSA" "AES" "RSA")

# iterate over the array with openssl ciphers -v
for CIPHER in "${CIPHERS[@]}"; do
    echo "Ciphers: '${CIPHER}'"
    openssl ciphers -V "${CIPHER}"
    echo ""
done


Using `openssl` to connect to a web site and display the certificate in an on-demand fashion.

> Note: Web sites can be load balanced across many IPv4 and IPv6 addresses.  This example merely accesses the first available.

In [ ]:
#!/bin/bash

#cipher_suite="\'ECDHE-ECDSA-AES128-SHA\'" # RSA key type
# cipher_suite='ECDHE-ECDSA-AES256-GCM-SHA384' # EC key type

!echo "[ OpenSSL - Connection Details ]============================"
!echo | openssl s_client -showcerts -cipher "ECDHE-ECDSA-AES128-SHA" -servername www.google.com -connect www.google.com:443 2>&1
!echo ""
!echo "[ OpenSSL - Certificate Details ]==========================="
!echo | openssl s_client -showcerts -cipher 'ECDHE-ECDSA-AES128-SHA' -servername www.google.com -connect www.google.com:443 2>&1 | openssl x509 -noout -text
